In [ ]:
import numpy as np
import pandas as pd
import math


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


.

.

. **SEQUENCE TO x2 SEQUENCE  experiments (initial)**

.

.
Tries to predict beginnings & endings of sound in audio sample using seq to (x2) seq model with GRU cells.

In [ ]:
data = pd.read_pickle('/content/drive/MyDrive/mreze/data/data/full_sequences.pkl')

In [ ]:
data.head()

,filename,mfccs,event_label,combine_squence_per_filename_and_event
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Dog,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Electric_shaver_toothbrush,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1238.wav,"[[-401.96198, -400.9539, -411.34525, -405.876,...",Blender,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
training = data[data['event_label'] == 'Alarm_bell_ringing']
X = np.array(training.mfccs.tolist())
y = np.array(training.combine_squence_per_filename_and_event.tolist())

X = np.einsum('ijk->ikj', X)

In [ ]:
def create_output_seq(y_orig):
  result_beginning = []
  result_end = []
  for sequence in y_orig:
    is_one = False
    new_sequence_beginning = []
    new_sequence_end = []
    to_assign_beginning = 0
    to_assign_end = 0
    for label in sequence:
      if label == 1:
        if is_one:
          to_assign_beginning = 0
        else:
          to_assign_beginning = 1
        is_one = True
      else:
        if is_one:
          new_sequence_end[len(new_sequence_end)-1] = 1
          to_assign_end = 0
        else:
          to_assign_end = 0
        is_one = False
      new_sequence_beginning.append(to_assign_beginning)
      new_sequence_end.append(to_assign_end)
    result_beginning.append(new_sequence_beginning)
    result_end.append(new_sequence_end)    
  return np.asarray(result_beginning), np.asarray(result_end)

In [ ]:
# Optimizovana sa paddingom (prosirenjem indikatora pocetka i kraja) - umesto jednog broja za pocetak i kraj -> (expansion_num * 2 +- 1) broj 
def create_output_seq_expanded(y_orig, expansion_num):
  result_beginning = []
  result_end = []
  for sequence in y_orig:
    new_sequence_beginning = [0] * len(sequence)
    new_sequence_end = [0] * len(sequence)
    indexes_beginning = [i for i in range(len(sequence)) if sequence[i] == 1 and i-1>=0 and i+1 < len(sequence) and sequence[i-1] == 0 and sequence[i+1] == 1] 
    indexes_end = [i for i in range(len(sequence)) if sequence[i] == 1 and i-1>=0 and i+1 < len(sequence) and sequence[i-1] == 1 and sequence[i+1] == 0] 

    for index in indexes_beginning:
      start = index - expansion_num if index - expansion_num >= 0 else index
      end = index + expansion_num if index + expansion_num < len(sequence) else index
      new_sequence_beginning[start:end] = [1] * (end - start)

    for index in indexes_end:
      start = index - expansion_num if index - expansion_num >= 0 else index
      end = index + expansion_num if index + expansion_num < len(sequence) else index
      new_sequence_end[start:end] = [1] * (end - start)

    result_beginning.append(new_sequence_beginning)
    result_end.append(new_sequence_end)    
  return np.asarray(result_beginning), np.asarray(result_end)

In [ ]:
y_new_beginning_exp, y_new_end_exp = create_output_seq_expanded(y, 2)
ind = 3
print(y[ind])
print(80*'-')
print(y_new_beginning_exp[ind])
print(80*'-')
print(y_new_end_exp[ind])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
--------------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
y_new_beginning, y_new_end = create_output_seq(y)

In [ ]:
# Check sequences
y_new_beginning[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
y_new_end[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# Show original
y[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
X.shape

(392, 431, 13)

In [ ]:
input_seq_len = 431
output_shape = 431
dense_layer_size = 150
num_of_features = 13
# output_len = len(data)

batch_size = 128  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 128  # Latent dimensionality of the encoding space.

In [ ]:
# Define architecture of autoencoder model. 
import tensorflow as tf
from tensorflow import keras

def create_encoder_decoder(input_seq_len = 431, num_of_features = 13):
  # Define an input sequence and process it.

  encoder_inputs = keras.Input(shape=(input_seq_len, num_of_features))
  encoder = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True)
  encoder_outputs, state_h = encoder(encoder_inputs)

  encoder_states = [state_h]

  decoder_beg = keras.layers.Dense(dense_layer_size, activation='relu')
  decoder_beg_outputs_0 = decoder_beg(encoder_outputs)

  decoder_beg_2 = keras.layers.Dense(1, activation='sigmoid')
  decoder_beg_outputs_2 = decoder_beg_2(decoder_beg_outputs_0)
  ###############################################################
  decoder_end = keras.layers.Dense(dense_layer_size, activation='relu')
  decoder_end_outputs_0 = decoder_end(encoder_outputs)

  # decoder_end_1 = keras.layers.Dense(dense_layer_size, activation='relu')
  # decoder_end_outputs_1 = decoder_end_1(decoder_end_outputs_0)

  decoder_end_2 = keras.layers.Dense(1, activation='sigmoid')
  decoder_end_outputs_2 = decoder_end_2(decoder_end_outputs_0)

  return encoder_inputs, decoder_beg_outputs_2, decoder_end_outputs_2
  


In [ ]:
encoder_inputs, decoder_outputs_beg, decoder_outputs_end = create_encoder_decoder()
model = keras.Model(encoder_inputs, [decoder_outputs_beg, decoder_outputs_end])
model.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 431, 13)]    0                                            
__________________________________________________________________________________________________
gru_14 (GRU)                    [(None, 431, 128), ( 54912       input_15[0][0]                   
__________________________________________________________________________________________________
dense_56 (Dense)                (None, 431, 150)     19350       gru_14[0][0]                     
__________________________________________________________________________________________________
dense_58 (Dense)                (None, 431, 150)     19350       gru_14[0][0]                     
___________________________________________________________________________________________

In [ ]:
y_new_beginning.shape

(392, 431)

In [ ]:
from keras.metrics import BinaryAccuracy, Precision, Recall, AUC
METRICS = [
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
      AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(
    optimizer="adam", loss=keras.losses.MeanSquaredError(), metrics=METRICS
)

model.fit(
    [X],
    [y_new_beginning, y_new_end],
    batch_size=batch_size,
    epochs=900,
    validation_split=0.2,
)
# model.save("/content/drive/MyDrive/mreze/models/s2c_300e_256b_allarm_bell_ringing")

In [ ]:
# USED ONLY FOR TESTING

# CONCLUSIONS: TOO MUCH FALSE NEGATIVES - BAD RECALL
#              BIGGER LATENT_TIM -> BETTER RESULTS -- LAT_DIM-F1 : 13-68, 64-70 256-73
def simple_evaluation(y_pred, y_test):
  tp = 0
  tn = 0
  fp = 0
  fn = 0

  for i, pred_arr in enumerate(y_pred):
    for j, pred_value in enumerate(pred_arr):
      predicted = pred_value
      real = y_test[i][j]
      if predicted and real:
        tp = tp + 1
      elif not(predicted) and real:
        fn = fn + 1
      elif predicted and not(real):
        fp = fp + 1
      elif not(predicted) and not(real):
        tn = tn + 1
    # break

  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = 2 * precision * recall / (precision + recall)

  print("PRECISION: " + str(precision))
  print("RECALL: " + str(recall))
  print("F1: " + str(f1))
  print("-"*20)
  print("tp: " + str(tp))
  print("tn: " + str(tn))
  print("fp: " + str(fp))
  print("fn: " + str(fn))

In [ ]:
def combine_beginning_end(y_beg, y_end):
  result = []
  for j in range(0, len(y_beg)):
    y_beg_curr = y_beg[j]
    y_end_curr = y_end[j]
    y_final = []
    was_one = False
    to_append_new = 0
    to_append = 0
    for i in range(0, len(y_beg_curr)):
      if y_beg_curr[i] == 1:
        if was_one:
          y_final[index_last_beginning:i] = [0] * (i - index_last_beginning)
        else:
          to_append_new = 1
          was_one = True
        index_last_beginning = i  
        
      if y_end_curr[i] == 1:
        if was_one:
          to_append_new = 0
          was_one = False
      y_final.append(to_append) 
      to_append = to_append_new 
    result.append(y_final)  
  return result 

In [ ]:
# Predict all
predicted_beg, predicted_end = model.predict(X)

In [ ]:
predicted_beg_rounded = [np.round(np.concatenate(i, axis=0), 0).astype(int) for i in predicted_beg]
predicted_end_rounded = [np.round(np.concatenate(i, axis=0), 0).astype(int) for i in predicted_end]

In [ ]:
simple_evaluation(predicted_beg_rounded,y_new_beginning)
simple_evaluation(predicted_end_rounded,y_new_end)

PRECISION: 0.7593688362919132
RECALL: 0.5099337748344371
F1: 0.6101426307448495
--------------------
tp: 385
tn: 168075
fp: 122
fn: 370
PRECISION: 0.3302063789868668
RECALL: 0.2646616541353383
F1: 0.2938230383973288
--------------------
tp: 176
tn: 167930
fp: 357
fn: 489


In [ ]:
# Best results so far
simple_evaluation(predicted_beg_rounded,y_new_beginning)
simple_evaluation(predicted_end_rounded,y_new_end)

PRECISION: 0.8018720748829953
RECALL: 0.680794701986755
F1: 0.7363896848137534
--------------------
tp: 514
tn: 168070
fp: 127
fn: 241
PRECISION: 0.47368421052631576
RECALL: 0.3383458646616541
F1: 0.3947368421052631
--------------------
tp: 225
tn: 168037
fp: 250
fn: 440


In [ ]:
final_combined = combine_beginning_end(predicted_beg_rounded, predicted_end_rounded)

In [ ]:
# This was run after the next one with differently trained model.
simple_evaluation(final_combined,y)

PRECISION: 0.36000400060009
RECALL: 0.4215735074517612
F1: 0.38836365598068706
--------------------
tp: 14398
tn: 109203
fp: 25596
fn: 19755


In [ ]:
# Best results so far
simple_evaluation(final_combined,y)

PRECISION: 0.44091962634326787
RECALL: 0.5514303282288525
F1: 0.49002159602425
--------------------
tp: 18833
tn: 110919
fp: 23880
fn: 15320
